In [ ]:
# imports + path
import os
import pandas as pd
from lxml import etree
path = '' # path to DATA folder in Lassy Groot (with \\ instead of \)
path_to_save = '' # path where csv file is to be saved (with \\ instead of \)

In [ ]:
# queries
om_query = '//node[@cat="oti"]'
te_query = '//node[@cat="ti" and parent::node[@cat="ssub" or @cat="smain" or @cat="sv1"]]'
niet_query = '//node[node[@word="niet"] and node[@cat="ti" and parent::node[@cat="ssub" or @cat="smain" or @cat="sv1"]]]'

In [ ]:
# data extraction using for loops and the queries
folders = os.listdir(path)
om_sentences = []
te_sentences = []
niet_sentences = []
for folder in folders:
    subpath = path + '\\' + folder + '\\ZIP'
    subfolders = os.listdir(subpath)
    for subfolder in subfolders:
        if (not subfolder.endswith('.zip')):
            path_to_files = subpath + '\\' + subfolder
            files = os.listdir(path_to_files)
            for file in files:
                if file.endswith('.xml'):
                    element_tree = etree.parse(path_to_files + '\\' + file)
                    sentence = element_tree.xpath('//sentence')[0].text
                    om_test = element_tree.xpath(om_query)
                    te_test = element_tree.xpath(te_query)
                    niet_test = element_tree.xpath(niet_query)
                    if om_test:
                        verb = element_tree.xpath('//node[@cat="oti"]/ancestor::node[@cat="smain" or @cat="ssub" or @cat="smain" or @cat="sv1"][1]/node[@wvorm="pv"]/@lemma')
                        if (len(verb) != 0) and (not sentence.startswith('Om ')):
                            sentence_type = element_tree.xpath('//node[@cat="oti"]/ancestor::node[@cat="smain" or @cat="ssub" or @cat="smain" or @cat="sv1"][1]/@cat')
                            subsection = element_tree.xpath('//node[@cat="oti"]/descendant::*/../node[@word]/@word')
                            subsection_positions = element_tree.xpath('//node[@cat="oti"]/descendant::*/../node[@word]/@begin')
                            sorted_subsection = [x for _, x in sorted(zip([int(i) for i in subsection_positions], subsection))]
                            sorted_subsection.insert(0, verb[0].replace('_', ''))
                            length_subsection = len(sorted_subsection)
                            relevant_subsection = " ".join(sorted_subsection).lower()
                            om_sentences.append((sentence, verb[0], sentence_type[0], relevant_subsection, length_subsection, file))
                    if te_test:
                        verb = element_tree.xpath('//node[@cat="ti"]/../node[@wvorm="pv"]/@lemma')
                        if (len(verb) != 0) and (not sentence.startswith('Te ')):
                            sentence_type = element_tree.xpath('//node[@cat="ti"]/parent::node[@cat="ssub" or @cat="smain" or @cat="sv1"]/@cat')
                            subsection = element_tree.xpath('//node[@cat="ti"]/descendant::*/../node[@word]/@word')
                            subsection_positions = element_tree.xpath('//node[@cat="ti"]/descendant::*/../node[@word]/@begin')
                            sorted_subsection = [x for _, x in sorted(zip([int(i) for i in subsection_positions], subsection))]
                            sorted_subsection.insert(0, verb[0].replace('_', ''))
                            length_subsection = len(sorted_subsection)
                            relevant_subsection = " ".join(sorted_subsection).lower()
                            te_sentences.append((sentence, verb[0], sentence_type[0], relevant_subsection, length_subsection, file))
                    if niet_test:
                        verb = element_tree.xpath('//node[@cat="ti"]/../node[@wvorm="pv"]/@lemma')
                        if (len(verb) != 0):
                            sentence_type = element_tree.xpath('//node[@cat="ti"]/parent::node[@cat="ssub" or @cat="smain" or @cat="sv1"]/@cat')
                            subsection = element_tree.xpath('//node[@cat="ti"]/descendant::*/../node[@word]/@word')
                            subsection_positions = element_tree.xpath('//node[@cat="ti"]/descendant::*/../node[@word]/@begin')
                            sorted_subsection = [x for _, x in sorted(zip([int(i) for i in subsection_positions], subsection))]
                            sorted_subsection.insert(0, verb[0].replace('_', ''))
                            sorted_subsection.insert(0, 'niet')
                            length_subsection = len(sorted_subsection)
                            relevant_subsection = " ".join(sorted_subsection).lower()
                            niet_sentences.append((sentence, verb[0], sentence_type[0], relevant_subsection, length_subsection, file))

In [ ]:
# creating and filling the DataFrame
col = ['Category', 'Sentence', 'Verb', 'Sentence Type', 'Relevant Subsection', 'Length Relevant Subsection', 'Filename']
df = pd.DataFrame(columns=col)

for sentence, verb, type, subsection, length, filename in om_sentences:
    df = pd.concat([df, pd.DataFrame([['oti', sentence, verb, type, subsection, length, filename]], columns=col)])

for sentence, verb, type, subsection, length, filename in te_sentences:
    df = pd.concat([df, pd.DataFrame([['ti', sentence, verb, type, subsection, length, filename]], columns=col)])

for sentence, verb, type, subsection, length, filename in niet_sentences:
    df = pd.concat([df, pd.DataFrame([['niet + ti', sentence, verb, type, subsection, length, filename]], columns=col)])

In [ ]:
# saving the DataFrame
df.to_csv(path_to_save + '\\sentences_Lassy_Groot.csv')